# Overview

This notebook provides a core set of analysis on a policy, primary focused on least-privilege and system self-protection. This is geared towards systems that have a customized SELinux policy based upon reference policy (and usually a RHEL or Fedora derivative).

To use this notebook you should:

   * Replace the policies loaded with the system policies that you are interested in.
   * Run each analysis and examine the output - this notebook doesn't give you yes/no answers. It requires interpretation.
   
# Getting the Policies

To use these notebook you need the source and binary versions of two policies: the system and base policy. Base policy, here, meaning the policy that the system developers started with. This is typically the policy from the version of RHEL or Fedora that the system is based upon.

System developers typically either add policy modules, directly modify the base policy, or both. Regardless of the approach, it's nice to have the base policy in both source and binary form.

Getting the source and binary policies for the system is dependent on how the system is developed.

The easiest way to get the source for the base policy is to start with the source for selinux-policy. You can grab those from:

   * http://vault.centos.org/
   * http://archives.fedoraproject.org/pub/archive/fedora/linux/updates/
   
Once you have the source rpm, you need to at least partially build it so that all of the patches are applied. First setup your system to build rpms - there is some info at https://wiki.centos.org/HowTos/RebuildSRPM. From there, just rpmbuild -bp will output the patched source to ~/rpmbuild/BUILD.

You can also build the binary policy this way, but it's actually kind of pain. It's easier to just grab the rpm and extract it with:

`rpm2cpio selinux-policy-targeted-3.13.1-190.fc24.noarch.rpm | cpio -idmv`

After you have done this, the policy should be under etc/selinux/targeted/policy/policy.30 (or similar) in the current directory.

Once you've collected the policies, simply save the source and binaries under the same directory with this notebook.

## Import and load the policies

You should change policy_paths.config to match the policies that you want to analyze.

As an example for this notebook, we are going to use a Fedora 25 policy as the system policy and a Fedora 24 policy as the base policy.

In [1]:
# Import senotebook - this complication is just to handle running this in the development tree
try:
    import span as se
except:
    import os
    path = os.path.dirname(os.getcwd())
    import sys
    sys.path.insert(0, path)
    import span as se

import pandas as pd
from IPython.display import display, HTML

# P - binary system policy
# ps - source system policy
# bp - binary base policy
# bps - source base policy
p, ps, bp, bps = se.load_policies_from_config("policy_paths.config")

# Basic Policy Summary

## Numer of Types

In [2]:
p_types = set(se.as_str(p.types()))
print("total types in custom policy: %d" % len(p_types))

p_domains = p.domain_types()
print("total number of domain types: %d" % len(p_domains))

total types in custom policy: 4792
total number of domain types: 820


## Custom Types

This is a quick way to figure out the _potentially_ custom types. It's only potentially, because it's possible that the base policy may have backported policy modules from newer policies and included them. So types not in the base policy would be present but not really custom to the system being analyzed.

Having said that, this approach is typically a good first approximation.

In [3]:
custom_types, custom_domains = p.new_types(bp)

print("total number of potentially custom types: %d" % len(custom_types))
print("total number of potentially custom domains: %d" % len(custom_domains))

total number of potentially custom types: 34
total number of potentially custom domains: 5


In [4]:
p.types_summary(custom_domains)

,name,attributes
0,container_t,"[corenet_unlabeled_type, domain, kernel_system_state_reader, mcs_constrained_type, netlabel_peer_type, nsswitch_domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, process_user_target, sandbox_caps_domain, sandbox_net_domain, svirt_sandbox_domain, syslog_client_type]"
1,hwloc_dhwd_t,"[application_domain_type, corenet_unlabeled_type, domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, systemprocess]"
2,ipa_ods_exporter_t,"[corenet_unlabeled_type, daemon, domain, ipa_domain, netlabel_peer_type, nsswitch_domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, syslog_client_type]"
3,sbd_t,"[corenet_unlabeled_type, daemon, domain, kernel_system_state_reader, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, syslog_client_type]"
4,systemd_modules_load_t,"[can_load_kernmodule, corenet_unlabeled_type, daemon, domain, kernel_system_state_reader, netlabel_peer_type, nsswitch_domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, systemd_domain]"


In [5]:
p.types_summary(custom_types)

,name,attributes
0,brltty_log_t,"[file_type, logfile, non_auth_file_type, non_security_file_type, sandbox_typeattr_1, sandbox_typeattr_2, sandbox_typeattr_3, sandbox_typeattr_4]"
1,container_file_t,"[device_node, file_type, filesystem_type, mountpoint, non_auth_file_type, non_security_file_type, noxattrfs, ptynode, sandbox_typeattr_1, sandbox_typeattr_2, sandbox_typeattr_3, sandbox_typeattr_4, svirt_file_type]"
2,container_ro_file_t,"[file_type, mountpoint, non_auth_file_type, non_security_file_type, sandbox_typeattr_1, sandbox_typeattr_2, sandbox_typeattr_3, sandbox_typeattr_4, svirt_file_type]"
3,container_t,"[corenet_unlabeled_type, domain, kernel_system_state_reader, mcs_constrained_type, netlabel_peer_type, nsswitch_domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, process_user_target, sandbox_caps_domain, sandbox_net_domain, svirt_sandbox_domain, syslog_client_type]"
4,fac_restore_client_packet_t,"[client_packet_type, packet_type]"
5,fac_restore_port_t,"[defined_port_type, port_type, unreserved_port_type]"
6,fac_restore_server_packet_t,"[packet_type, server_packet_type]"
7,hsa_device_t,[device_node]
8,hwloc_dhwd_exec_t,"[application_exec_type, entry_type, exec_type, file_type, non_auth_file_type, non_security_file_type, systemprocess_entry]"
9,hwloc_dhwd_t,"[application_domain_type, corenet_unlabeled_type, domain, pcmcia_typeattr_1, pcmcia_typeattr_2, pcmcia_typeattr_3, pcmcia_typeattr_4, pcmcia_typeattr_5, pcmcia_typeattr_6, pcmcia_typeattr_7, systemprocess]"


## Initial SIDs
This is the list of initial security identifiers for object classes present in this policy.

In [6]:
def initsids_as_dataframe(p):
    initsids = [{ "name": str(x), "context": str(x.context)} for x in p.initialsids()]
    return pd.DataFrame(initsids)[["name", "context"]]
    
initsids_as_dataframe(p)

,name,context
0,kernel,system_u:system_r:kernel_t:s0
1,security,system_u:object_r:security_t:s0
2,unlabeled,system_u:object_r:unlabeled_t:s0
3,fs,system_u:object_r:fs_t:s0
4,file,system_u:object_r:unlabeled_t:s0
5,file_labels,system_u:object_r:unlabeled_t:s0
6,init,system_u:object_r:unlabeled_t:s0
7,any_socket,system_u:object_r:unlabeled_t:s0
8,port,system_u:object_r:port_t:s0
9,netif,system_u:object_r:netif_t:s0


In [7]:
p_initsids = {str(x): str(x.context) for x in p.initialsids()}
b_initsids = {str(x): str(x.context) for x in bp.initialsids()}

Any new or missing initial sids?

In [8]:
new_initsids = set(p_initsids.keys()) - set(b_initsids.keys())
print("new initial sids: " + str(new_initsids))

missing_initsids = set(b_initsids.keys()) - set(p_initsids.keys())
print("missing initial sids: " + str(missing_initsids))

new initial sids: set()
missing initial sids: set()


Changed labels?

In [9]:
changed_initsids = []
for i in p_initsids.keys():
    if not i in b_initsids:
        changed_initsids.append({"name": i, "system": p_initsids[i], "base": "NOT PRESENT"})
        continue
    if p_initsids[i] != b_initsids[i]:
        changed_initsids.append({"name": i, "system": p_initsids[i], "base": b_initsids[i]})
        
if len(changed_initsids) > 0:
    display(pd.DataFrame(changed_initsids)[["name", "system", "base"]])
else:
    print("No changed initial sid labels")


No changed initial sid labels


## Object Classes
This is the list of new object classes, both kernel and userspace, present in the policy.

In [10]:
se.as_strset(p.classes()) - se.as_strset(bp.classes())

{'cap2_userns', 'cap_userns'}

New permissions or missing permissions:

In [11]:
# We are going to do commons separately becase they can be named the same as
# object classes
p_commons = {str(x): se.as_strset(x.perms) for x in p.commons()}
p_classes = {str(x): se.as_strset(x.perms) for x in p.classes()}
b_commons = {str(x): se.as_strset(x.perms) for x in bp.commons()}
b_classes = {str(x): se.as_strset(x.perms) for x in bp.classes()}

print("new commons: " + str(set(p_commons.keys()) - set(b_commons.keys())))
print("new object classes: " + str(set(p_classes.keys()) - set(b_classes.keys())))
print("removed object classes: " + str(set(b_classes.keys()) - set(p_classes.keys())))
print("removed commons: " + str(set(b_commons.keys()) - set(p_commons.keys())))


new commons: {'cap2', 'cap'}
new object classes: {'cap_userns', 'cap2_userns'}
removed object classes: set()
removed commons: set()


Changed permissions:

In [12]:
def find_object_changes(name, pc, bc):
    cc = []
    for c in pc:
        if not c in bc:
            cc.append({"name": c, "added": str(pc[c]), "removed": "New Class"})
            continue
        added = pc[c] - bc[c]
        removed = bc[c] - pc[c]

        if len(added) or len(removed):
            cc.append({"name": c, "added": str(added), "removed": str(removed)})

    if len(cc) > 0:
        print("Changed %s permissions:" % name)
        display(pd.DataFrame(cc)[["name", "added", "removed"]])
    else:
        print("No changed %s permissions." % name)
        
find_object_changes("common", p_commons, b_commons)
find_object_changes("object class", p_classes, b_classes)
    

Changed common permissions:


,name,added,removed
0,cap,"{'sys_rawio', 'net_raw', 'audit_control', 'fowner', 'setfcap', 'net_broadcast', 'mknod', 'sys_nice', 'dac_override', 'fsetid', 'setuid', 'setgid', 'sys_time', 'dac_read_search', 'linux_immutable', 'sys_pacct', 'sys_module', 'setpcap', 'ipc_lock', 'ipc_owner', 'lease', 'net_bind_service', 'chown', 'sys_tty_config', 'audit_write', 'sys_chroot', 'sys_ptrace', 'kill', 'sys_admin', 'sys_boot', 'sys_resource', 'net_admin'}",New Class
1,cap2,"{'wake_alarm', 'audit_read', 'syslog', 'mac_override', 'mac_admin', 'block_suspend'}",New Class


Changed object class permissions:


,name,added,removed
0,process,set(),{'ptrace_child'}
1,capability2,set(),"{'wake_alarm', 'audit_read', 'syslog', 'mac_admin', 'compromise_kernel', 'mac_override', 'block_suspend'}"
2,cap_userns,set(),New Class
3,capability,set(),"{'sys_nice', 'net_admin', 'dac_override', 'setuid', 'sys_pacct', 'ipc_lock', 'ipc_owner', 'net_bind_service', 'audit_write', 'sys_ptrace', 'kill', 'sys_admin', 'sys_resource', 'sys_rawio', 'net_raw', 'audit_control', 'fowner', 'setfcap', 'mknod', 'fsetid', 'setgid', 'sys_time', 'dac_read_search', 'linux_immutable', 'sys_module', 'setpcap', 'lease', 'chown', 'sys_tty_config', 'sys_chroot', 'sys_boot', 'net_broadcast'}"
4,cap2_userns,set(),New Class
5,system,"{'start', 'stop'}",set()


## Constraints
Since analysis tools do not address constraints in a meaningful way, here is a raw dump of the constraints for reference.

In [13]:
HTML(ps.diff_mls_constraints(bps))

In [14]:
HTML(ps.diff_mcs_constraints(bps))

In [15]:
HTML(ps.diff_constraints(bps))

## Policy Caps

Policy capabilities control how policy enforcement is done on a system and is used to control optional features and backwards compatibility. Some documentation can be found at https://selinuxproject.org/page/Policy_Configuration_Statements.

In [16]:
se.as_str(p.polcaps())

['network_peer_controls', 'open_perms']

Changed policy capabilities:

In [17]:
se.as_strset(p.polcaps()) ^ se.as_strset(bp.polcaps())

set()

# System Self Protection Analysis

Analyses of access that can compromise the ability of the system to protect itself from malicious processes.

## Write Rules - boot_t
Files labeled boot_t include files for bootstrapping the system, including the critical kernel image and initial ramdisk image. Domains that can modify these files can completely compromise a system by replacing the kernel or initrd.

In [18]:
print(ps.file_contexts("boot_t"))

./kernel/files.fc:9:/initrd\.img.*		-l	gen_context(system_u:object_r:boot_t,s0)
./kernel/files.fc:10:/vmlinuz.*		-l	gen_context(system_u:object_r:boot_t,s0)
./kernel/files.fc:31:/boot				gen_context(system_u:object_r:boot_t,s0)
./kernel/files.fc:32:/boot/.*			gen_context(system_u:object_r:boot_t,s0)



In [19]:
p.object_info_flow(object_type="boot_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,amtu_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
2,bootloader_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
3,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
4,initrc_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
5,kdumpctl_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
6,kdumpgui_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
7,pegasus_openlmi_logicalfile_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
8,restorecond_t,None,"{open, lock, getattr, read, ioctl, relabelto, relabelfrom}"
9,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"


## Read Rules - system_map_t
Files labeled system_map_t contain the kernel symbol table which can be used to lookup the address of symbols in the kernel to aid in exploitation of the system. 

In [20]:
print(ps.file_contexts("system_map_t"))

./kernel/files.fc:34:/boot/efi(/.*)?/System\.map(-.*)? -- gen_context(system_u:object_r:system_map_t,s0)
./kernel/files.fc:37:/boot/System\.map(-.*)?	--	gen_context(system_u:object_r:system_map_t,s0)



In [21]:
p.object_info_flow(object_type="system_map_t", direction="r")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,abrt_t,None,"{open, lock, getattr, read, ioctl}"
2,aide_t,None,"{open, lock, getattr, read, ioctl}"
3,amanda_t,None,"{open, lock, getattr, read, ioctl}"
4,bacula_t,None,"{open, lock, getattr, read, ioctl}"
5,bootloader_t,None,"{open, lock, getattr, read, ioctl}"
6,cgred_t,None,"{read, getattr}"
7,cluster_t,None,"{open, lock, getattr, read, ioctl}"
8,collectd_t,None,"{open, lock, getattr, read, ioctl}"
9,container_t,None,"{open, lock, getattr, read, ioctl}"


## Write Rules - modules_object_t
modules_object_t is the label placed on kernel modules. Files with this label contain kernel code that, depending on other aspects of system configuration, can be automatically loaded into the kernel as needed. Domains that can write to these files can compromise the integrity of the system.

In [22]:
p.object_info_flow(object_type="modules_object_t", direction="w", tclass=["file", "dir"])

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, reparent, open, lock, unlink, remove_name, add_name, rmdir, link, create, write, getattr, append, read, search}"
1,automount_t,None,"{reparent, rename, rmdir, open, setattr, lock, create, write, link, getattr, read, ioctl, search, unlink, remove_name, add_name}"
2,bootloader_t,None,"{rename, setattr, ioctl, add_name, open, link, lock, create, write, getattr, append, read, search, unlink, remove_name}"
3,depmod_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
4,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, reparent, open, lock, execute, unlink, remove_name, relabelfrom, quotaon, audit_access, mounton, add_name, execute_no_trans, rmdir, swapon, link, create, write, getattr, append, read, search, relabelto}"
5,glusterd_t,None,"{mounton, write, setattr}"
6,insmod_t,None,"{rename, setattr, ioctl, add_name, open, link, lock, create, write, getattr, append, read, search, unlink, remove_name}"
7,neutron_t,None,"{mounton, write, setattr}"
8,pegasus_openlmi_logicalfile_t,None,"{rename, setattr, ioctl, reparent, open, lock, unlink, remove_name, add_name, rmdir, link, create, write, getattr, append, read, search}"
9,prelink_t,None,{write}


## Read/Write Rules - memory_device_t
Files labeled memory_device_t provide access to the raw physical memory of the solution. As such, reading from these files can compromise the confidentiality of the solution. Writing to these files can compromise the integrity of the solution.

In [23]:
p.terules_query(target="memory_device_t", perms=["read"], tclass=["chr_file"])

,source,target,tclass,perms,cond
0,abrt_t,memory_device_t,chr_file,"{open, lock, getattr, read, ioctl}",None
1,colord_t,memory_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
2,devices_unconfined_type,device_node,chr_file,"{execute_no_trans, rename, open, swapon, setattr, lock, create, write, link, getattr, execute, append, read, quotaon, ioctl, audit_access, relabelto, mounton, unlink, relabelfrom}",None
3,dmesg_t,memory_device_t,chr_file,"{open, lock, getattr, read, ioctl}",None
4,dmidecode_t,memory_device_t,chr_file,"{open, lock, getattr, read, ioctl}",None
5,freeipmi_bmc_watchdog_t,memory_device_t,chr_file,"{open, lock, getattr, read, ioctl}",None
6,freeipmi_ipmiseld_t,memory_device_t,chr_file,"{open, lock, getattr, read, ioctl}",None
7,init_t,memory_device_t,chr_file,"{open, lock, getattr, read, ioctl}",None
8,initrc_t,memory_device_t,chr_file,"{open, lock, write, getattr, execute, append, read, ioctl}",None
9,kdumpctl_t,device_node,chr_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}",None


In [24]:
p.terules_query(target="memory_device_t", perms=["write", "append"], tclass=["chr_file"])

,source,target,tclass,perms,cond
0,colord_t,memory_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
1,devices_unconfined_type,device_node,chr_file,"{execute_no_trans, rename, open, swapon, setattr, lock, create, write, link, getattr, execute, append, read, quotaon, ioctl, audit_access, relabelto, mounton, unlink, relabelfrom}",None
2,initrc_t,memory_device_t,chr_file,"{open, lock, write, getattr, execute, append, read, ioctl}",None
3,kdumpctl_t,device_node,chr_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}",None
4,rpm_script_t,device_node,chr_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",None
5,rpm_t,device_node,chr_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}",None
6,rtas_errd_t,memory_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
7,sandbox_domain,device_node,chr_file,"{lock, write, getattr, append, read, ioctl}",None
8,systemd_tmpfiles_t,device_node,chr_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}",None
9,udev_t,device_node,chr_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}",None


## Read/Write Rules - proc_kcore_t
Similar to files labeled proc_kcore_t provide access to the virtual memory of the solution. As such, reading from these files can compromise the confidentiality of the solution. Writing to these files can compromise the integrity of the solution.

In [25]:
print(ps.genfscon("proc"))

./kernel/devices.te:204:genfscon proc /mtrr gen_context(system_u:object_r:mtrr_device_t,s0)
./kernel/files.te:156:genfscon proc /kallsyms gen_context(system_u:object_r:system_map_t,s0)
./kernel/kernel.te:85:genfscon proc / gen_context(system_u:object_r:proc_t,s0)
./kernel/kernel.te:86:genfscon proc /sysvipc gen_context(system_u:object_r:proc_t,s0)
./kernel/kernel.te:89:genfscon proc /fs/openafs gen_context(system_u:object_r:proc_afs_t,s0)
./kernel/kernel.te:93:genfscon proc /kmsg gen_context(system_u:object_r:proc_kmsg_t,mls_systemhigh)
./kernel/kernel.te:99:genfscon proc /kcore gen_context(system_u:object_r:proc_kcore_t,mls_systemhigh)
./kernel/kernel.te:102:genfscon proc /mdstat gen_context(system_u:object_r:proc_mdstat_t,s0)
./kernel/kernel.te:105:genfscon proc /numatools gen_context(system_u:object_r:proc_numa_t,s0)
./kernel/kernel.te:109:genfscon proc /net gen_context(system_u:object_r:proc_net_t,s0)
./kernel/kernel.te:112:genfscon proc /sys/fs/protected_hardlinks gen_context(syst

In [26]:
p.terules_query(target="proc_kcore_t", perms=["read"], tclass=["file"])

,source,target,tclass,perms,cond
0,abrt_t,proc_type,file,"{open, lock, getattr, read, ioctl}",None
1,collectd_t,proc_type,file,"{open, lock, getattr, read, ioctl}",None
2,container_t,proc_type,file,"{open, lock, getattr, read, ioctl}",None
3,gpg_agent_t,proc_kcore_t,file,"{open, lock, getattr, read, ioctl}",None
4,kdump_t,proc_kcore_t,file,"{open, lock, getattr, read, ioctl}",None
5,kern_unconfined,proc_type,file,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, quotaon, relabelfrom, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}",None
6,snmpd_t,proc_type,file,"{open, lock, getattr, read, ioctl}",None
7,svirt_kvm_net_t,proc_type,file,"{open, lock, getattr, read, ioctl}",None
8,svirt_qemu_net_t,proc_type,file,"{open, lock, getattr, read, ioctl}",None
9,sysadm_t,proc_type,file,"{open, lock, getattr, read, ioctl}",None


In [27]:
p.terules_query(target="proc_kcore_t", perms=["write", "append"], tclass=["file"])

,source,target,tclass,perms,cond
0,kern_unconfined,proc_type,file,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, quotaon, relabelfrom, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}",None


## Read/Write Rules - Terminal Types
Terminal files represent a large information flow between domains that can read/write from these possibly shared devices. One example - a domain writes a password prompt to another terminal and then reads the data entered by a user.

### Read Terminals

In [28]:
p.terules_query(target=["console_device_t", "tty_device_t", "ptmx_t", "devpts_t"], perms=["read"], tclass=["chr_file"])

,source,target,tclass,perms,cond
0,NetworkManager_t,tty_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
1,ajaxterm_t,ptmx_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
2,amanda_t,file_type,chr_file,"{open, lock, getattr, read, ioctl}",None
3,apcupsd_t,console_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
4,apcupsd_t,devpts_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
5,apcupsd_t,tty_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
6,apm_t,console_device_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None
7,apm_t,devpts_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None
8,apm_t,tty_device_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None
9,auditctl_t,console_device_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None


### Write Terminals

In [29]:
p.terules_query(target=["console_device_t", "tty_device_t", "ptmx_t", "devpts_t"], perms=["write", "append", "ioctl"], tclass=["chr_file"])

,source,target,tclass,perms,cond
0,NetworkManager_t,tty_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
1,ajaxterm_t,ptmx_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
2,amanda_t,file_type,chr_file,"{open, lock, getattr, read, ioctl}",None
3,apcupsd_t,console_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
4,apcupsd_t,devpts_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
5,apcupsd_t,tty_device_t,chr_file,"{open, lock, write, getattr, append, read, ioctl}",None
6,apm_t,console_device_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None
7,apm_t,devpts_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None
8,apm_t,tty_device_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None
9,auditctl_t,console_device_t,chr_file,"{lock, write, getattr, append, read, ioctl}",None


## Write Rules - etc_t
Files labeled etc_t includes a variety of files for system-wide configuration. In addition, default poilicies often fail to split out sensitive files into separate types outside of etc_t. One example that can impact stateless systems is rwtab and statetab. Modifications to these files can alter what state is retained across reboots.

On solutions prior to RHEL 7, the default label for the password file /etc/passwd us also etc_t, thus any domain that can write to etc_t can modify the file containing key userdata (though no longer password hashes), e.g., changing an unpriv user ID to UID 0.

In [30]:
p.object_info_flow(object_type="etc_t", tclass=["file", "dir"], direction="w")

,type,conditional,perms
0,NetworkManager_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
1,abrt_dump_oops_t,None,"{rename, setattr, ioctl, reparent, open, lock, unlink, remove_name, add_name, rmdir, link, create, write, getattr, append, read, search}"
2,alsa_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
3,apcupsd_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
4,automount_t,None,"{reparent, rename, rmdir, open, setattr, lock, create, write, link, getattr, read, ioctl, search, unlink, remove_name, add_name}"
5,avahi_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
6,bootloader_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
7,cardmgr_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
8,chfn_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"
9,cobblerd_t,None,"{open, lock, write, getattr, read, ioctl, search, remove_name, add_name}"


## Write Rules - system_conf_t
Files labeled sys_conf_t are applied to system-wide configuration files, such as those residing in /etc/sysconfig. Included in that set, are the iptables configuration files. Domains with the ability to write to these files can alter the firewall ruleset, including SELinux packet labels applied via SECMARK, in effect, providing the ability to alter the intent of the policy.

In [31]:
p.terules_query(target="system_conf_t", perms=["write", "append", "unlink"], tclass=["file"])

,source,target,tclass,perms,cond
0,abrt_dump_oops_t,non_security_file_type,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",None
1,files_unconfined_type,file_type,file,"{execute_no_trans, rename, open, swapon, setattr, lock, create, write, link, getattr, execute, append, read, quotaon, ioctl, audit_access, relabelto, mounton, unlink, relabelfrom}",None
2,firewallgui_t,system_conf_t,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",None
3,ftpd_t,non_security_file_type,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",ftpd_full_access
4,glusterd_t,non_security_file_type,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",gluster_export_all_rw
5,initrc_t,system_conf_t,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",None
6,iptables_t,system_conf_t,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",None
7,kernel_t,non_security_file_type,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}",nfs_export_all_rw
8,mount_t,non_security_file_type,file,"{read, write}",mount_anyfile
9,ncftool_t,system_conf_t,file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink}",None


## Write Rules - net_conf_t
Files labeled net_conf_t are applied to network configuration files. These files contain the information the system uses to config network devices. Domains that can modify these files can alter the configuration of devices such  as eth0.

In [32]:
p.object_info_flow(object_type="net_conf_t", direction="w")

,type,conditional,perms
0,NetworkManager_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
2,avahi_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
3,cardmgr_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
4,cobblerd_t,None,"{open, lock, write, getattr, append, read, ioctl}"
5,denyhosts_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
6,dhcpc_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
7,dnssec_trigger_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
8,fail2ban_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
9,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"


## Write Rules - lib_t
Files labeled lib_t are used by dynamically linked applications. A vast number of shared libraries are labeled lib_t. Domains that can modify these files can alter the behavior of any application linked against these files.

In [33]:
p.object_info_flow(object_type="lib_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
2,ldconfig_t,None,"{rename, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
3,pegasus_openlmi_logicalfile_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
4,prelink_t,None,"{rename, open, setattr, lock, create, write, link, getattr, execute, append, read, ioctl, relabelto, unlink, execmod, relabelfrom}"
5,restorecond_t,None,"{open, lock, getattr, read, ioctl, relabelto, relabelfrom}"
6,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
7,rpm_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
8,secadm_t,None,"{getattr, relabelto, relabelfrom}"
9,setfiles_mac_t,None,"{getattr, relabelto, relabelfrom}"


## Read/Write Rules - auditd_log_t var_log_t
Files labeled auditd_log_t contain the audit messages generated by the audit frameworks which is critical to monitoring the security posture of the solution. Files labeled var_log_t are typically files that should have a different label as var_log_t is a generic label and many base system domains have the ability to modify files with that label. Modifying these files can allow a domain to report mis-information or truncate otherwise sensitive log and audit entries.

In [34]:
# Write Audit logs
p.object_info_flow(object_type="auditd_log_t", direction="w")

,type,conditional,perms
0,auditadm_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
1,auditd_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
2,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
3,logadm_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
4,restorecond_t,None,"{open, lock, getattr, read, ioctl, relabelto, relabelfrom}"
5,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
6,rpm_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
7,sandbox_domain,None,"{lock, write, getattr, append, read, ioctl}"
8,secadm_t,None,"{getattr, relabelto, relabelfrom}"
9,setfiles_mac_t,None,"{getattr, relabelto, relabelfrom}"


In [35]:
# Read object logs
p.object_info_flow(object_type="auditd_log_t", direction="r")

,type,conditional,perms
0,aide_t,None,"{open, lock, getattr, read, ioctl}"
1,amanda_t,None,"{open, lock, getattr, read, ioctl}"
2,auditadm_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
3,auditd_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
4,bacula_t,None,"{open, lock, getattr, read, ioctl}"
5,cgred_t,None,"{read, getattr}"
6,fail2ban_client_t,None,"{open, lock, getattr, read, ioctl}"
7,fail2ban_t,None,"{open, lock, getattr, read, ioctl}"
8,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
9,logadm_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"


In [36]:
# Write generic logs
p.object_info_flow(object_type="var_log_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,application_domain_type,None,"{lock, append, ioctl, getattr}"
2,bootloader_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
3,daemon,None,"{lock, append, ioctl, getattr}"
4,dmesg_t,None,"{open, lock, write, getattr, append, ioctl}"
5,dovecot_deliver_t,None,"{open, lock, getattr, append, ioctl}"
6,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
7,httpd_sys_script_t,None,"{lock, append, ioctl, getattr}"
8,init_t,None,"{open, lock, write, getattr, append, read, ioctl}"
9,initrc_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"


## Read/Write Rules - shadow_t
Files labeled shadow_t contain user password data, including salts and hashes. Domains that can write to these files can modify user passwords.

Domains that can read from these files can collect user password data for use in, e.g., brute force attacks.

In [37]:
p.object_info_flow(object_type="shadow_t", tclass=["file"], direction="w")

,type,conditional,perms
0,cockpit_session_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
1,files_unconfined_type,None,"{execute_no_trans, rename, open, swapon, setattr, lock, create, write, link, relabelfrom, getattr, execute, append, read, ioctl, audit_access, relabelto, mounton, unlink, quotaon}"
2,groupadd_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}"
3,passwd_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}"
4,pegasus_openlmi_account_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}"
5,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
6,rpm_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
7,sandbox_domain,None,"{lock, write, getattr, append, read, ioctl}"
8,secadm_t,None,"{getattr, relabelto, relabelfrom}"
9,setfiles_mac_t,None,"{getattr, relabelto, relabelfrom}"


In [38]:
p.object_info_flow(object_type="shadow_t", tclass=["file"], direction="r")

,type,conditional,perms
0,accountsd_t,None,"{open, lock, getattr, read, ioctl}"
1,aide_t,None,"{open, lock, getattr, read, ioctl}"
2,amanda_t,None,"{open, lock, getattr, read, ioctl}"
3,bacula_t,None,"{open, lock, getattr, read, ioctl}"
4,chkpwd_t,None,"{open, lock, getattr, read, ioctl}"
5,cockpit_session_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
6,files_unconfined_type,None,"{execute_no_trans, rename, open, swapon, setattr, lock, create, write, link, relabelfrom, getattr, execute, append, read, ioctl, audit_access, relabelto, mounton, unlink, quotaon}"
7,groupadd_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}"
8,passwd_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}"
9,pegasus_openlmi_account_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}"


## Read/Write Rules - unlabeled_t Files
unlabeled_t is the default label for a number of object classes, such as packets and files. Almost nothing should have access to this type.

In [39]:
p.object_info_flow(object_type="unlabeled_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,afs_t,None,"{open, lock, write, getattr, append, read, ioctl}"
2,bootloader_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
3,ccs_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
4,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
5,fsadm_t,None,"{open, lock, write, getattr, append, read, ioctl}"
6,kern_unconfined,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto, entrypoint}"
7,pegasus_openlmi_logicalfile_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
8,restorecond_t,None,"{open, lock, getattr, read, ioctl, relabelto, relabelfrom}"
9,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"


In [40]:
p.object_info_flow(object_type="unlabeled_t", direction="r")

,type,conditional,perms
0,NetworkManager_t,None,"{open, lock, getattr, read, ioctl}"
1,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
2,afs_t,None,"{open, lock, write, getattr, append, read, ioctl}"
3,aide_t,None,"{open, lock, getattr, read, ioctl}"
4,amanda_t,None,"{open, lock, getattr, read, ioctl}"
5,bacula_t,None,"{open, lock, getattr, read, ioctl}"
6,bootloader_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
7,ccs_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
8,cgred_t,None,"{read, getattr}"
9,cluster_t,None,"{open, lock, getattr, read, ioctl}"


## Write Rules - wtmp_t
wtmp contains login data - another component of overall system logging and auditing. Writing to this allows one to modify the login records of the system.

In [41]:
p.object_info_flow(object_type="wtmp_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,acct_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
2,application_domain_type,None,"{lock, append, ioctl, getattr}"
3,audisp_remote_t,None,"{open, lock, getattr, append, ioctl}"
4,auditadm_su_t,None,"{open, lock, getattr, append, read, ioctl}"
5,chfn_t,None,"{open, lock, getattr, append, read, ioctl}"
6,cockpit_session_t,None,"{open, lock, write, getattr, append, read, ioctl}"
7,consolekit_t,None,"{open, lock, write, getattr, append, ioctl}"
8,daemon,None,"{lock, append, ioctl, getattr}"
9,dirsrv_t,None,"{open, lock, getattr, append, read, ioctl}"


## Write Rules - shell_exec_t
Shell binaries are relied upon by numerous system components. The ability to modify these binaries represents significant risk.

In [42]:
p.object_info_flow(object_type="shell_exec_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
2,pegasus_openlmi_logicalfile_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
3,prelink_t,None,"{rename, open, setattr, lock, create, write, link, getattr, execute, append, read, ioctl, relabelto, unlink, relabelfrom}"
4,restorecond_t,None,"{open, lock, getattr, read, ioctl, relabelto, relabelfrom}"
5,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
6,rpm_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
7,secadm_t,None,"{getattr, relabelto, relabelfrom}"
8,setfiles_mac_t,None,"{getattr, relabelto, relabelfrom}"
9,setfiles_t,None,"{getattr, read, relabelto, relabelfrom}"


## Write Rules - bin_t
Binaries labeled bin_t are relied upon by numerous system components. The ability to modify these binaries represents significant risk.

In [43]:
p.object_info_flow(object_type="bin_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
2,pegasus_openlmi_logicalfile_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
3,prelink_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, execute, append, read, relabelto, unlink, relabelfrom}"
4,restorecond_t,None,"{open, lock, getattr, read, ioctl, relabelto, relabelfrom}"
5,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
6,rpm_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
7,secadm_t,None,"{getattr, relabelto, relabelfrom}"
8,setfiles_mac_t,None,"{getattr, relabelto, relabelfrom}"
9,setfiles_t,None,"{getattr, read, relabelto, relabelfrom}"


## Read/Write Rules - fixed_disk_device_t
Fixed storage devices e.g., HDDs, should have strictly controlled raw write access, or write access in general. Writing to fixed disk devicces can compromise the integrity of the system. Reading from fixed disk devices can compromise the confidentiality of data residing on the system.

In [44]:
# Write to fixed disk devices
p.terules_query(target="fixed_disk_device_t", perms=["write"], tclass=["blk_file"])

,source,target,tclass,perms,cond
0,bootloader_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None
1,clogd_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None
2,clvmd_t,fixed_disk_device_t,blk_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}",None
3,cmirrord_t,fixed_disk_device_t,blk_file,"{open, lock, create, write, getattr, read, ioctl}",None
4,devicekit_disk_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None
5,devices_unconfined_type,device_node,blk_file,"{rename, open, swapon, setattr, lock, create, write, link, getattr, execute, append, read, quotaon, ioctl, audit_access, relabelto, mounton, unlink, execmod, relabelfrom}",None
6,drbd_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None
7,fenced_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None
8,fsadm_t,fixed_disk_device_t,blk_file,"{open, swapon, lock, write, getattr, append, read, ioctl}",None
9,fsdaemon_t,fixed_disk_device_t,blk_file,"{open, lock, create, write, getattr, append, read, ioctl}",None


In [45]:
# Read from fixed disk devices
p.terules_query(target="fixed_disk_device_t", perms=["read"], tclass=["blk_file"])

,source,target,tclass,perms,cond
0,amanda_t,fixed_disk_device_t,blk_file,"{open, lock, getattr, read, ioctl}",None
1,bacula_t,fixed_disk_device_t,blk_file,"{open, lock, getattr, read, ioctl}",None
2,blkmapd_t,fixed_disk_device_t,blk_file,"{open, lock, getattr, read, ioctl}",None
3,bootloader_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None
4,clogd_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None
5,cloud_init_t,fixed_disk_device_t,blk_file,"{open, lock, getattr, read, ioctl}",None
6,cluster_t,fixed_disk_device_t,blk_file,"{open, lock, getattr, read, ioctl}",None
7,clvmd_t,fixed_disk_device_t,blk_file,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, relabelto, unlink, relabelfrom}",None
8,cmirrord_t,fixed_disk_device_t,blk_file,"{open, lock, create, write, getattr, read, ioctl}",None
9,devicekit_disk_t,fixed_disk_device_t,blk_file,"{open, lock, write, getattr, append, read, ioctl}",None


## Write Rules - default_t
Anything labeled default_t is in indication a more-specific label does not exist, which means there is a problem with the policy.

In [46]:
p.object_info_flow(object_type="default_t", direction="w")

,type,conditional,perms
0,abrt_dump_oops_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
1,files_unconfined_type,None,"{rename, setattr, ioctl, execmod, open, lock, execute, unlink, relabelfrom, quotaon, audit_access, mounton, execute_no_trans, swapon, link, create, write, getattr, append, read, relabelto}"
2,pegasus_openlmi_logicalfile_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, unlink}"
3,restorecond_t,None,"{open, lock, getattr, read, ioctl, relabelto, relabelfrom}"
4,ricci_modstorage_t,None,"{rename, open, setattr, lock, create, write, link, getattr, append, read, ioctl, unlink}"
5,rpm_script_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
6,rpm_t,None,"{rename, setattr, ioctl, open, link, lock, create, write, getattr, append, read, relabelto, unlink, relabelfrom}"
7,sandbox_domain,None,"{lock, write, getattr, append, read, ioctl}"
8,secadm_t,None,"{getattr, relabelto, relabelfrom}"
9,setfiles_mac_t,None,"{getattr, relabelto, relabelfrom}"


## Transition - initrc_t
On versions <= RHEL 6, initrc_t is a very powerful domain. Domains that can transition to initrc_t can start, stop, or otherwise influence services running on the system.

In [47]:
p.domains_with(target_name="initrc_t", tclass=["process"], perms=["transition"])

,type,conditional,perms
0,NetworkManager_t,None,"{sigkill, transition, signal, signull}"
1,anaconda_t,None,{transition}
2,apmd_t,None,{transition}
3,authconfig_t,None,"{transition, sigchld}"
4,certmonger_unconfined_t,None,{transition}
5,cluster_t,None,"{transition, getattr}"
6,cobblerd_t,None,{transition}
7,condor_startd_t,None,{transition}
8,crond_t,None,{transition}
9,ctdbd_t,None,{transition}


## Memory Permissions
These permissions control access to specific operations on memory segments. Granting these permissions makes it easier to exploit a vulnerable process and should be avoided. Most can be addressed via code or compilation flag changes.

In [48]:
p.domains_with(target_name="", tclass=["process"], perms=["execmem", "execmod", "execheap", "execstack"])

,type,conditional,perms
0,anaconda_t,None,"{sigkill, signal, getsched, sigstop, dyntransition, execmem, transition, fork, signull, getcap, sigchld}"
1,blueman_t,None,"{sigkill, signal, getsched, sigstop, execmem, setsched, fork, signull, getcap, sigchld}"
2,boinc_project_t,None,"{sigkill, signal, getsched, sigstop, setcap, dyntransition, setsched, transition, setpgid, fork, signull, execstack, getcap, sigchld}"
3,bootloader_t,None,"{sigkill, signal, getsched, sigstop, dyntransition, execmem, transition, fork, signull, getcap, sigchld}"
4,chrome_sandbox_nacl_t,None,"{sigkill, signal, getsched, sigstop, execmem, setsched, fork, signull, getcap, sigchld}"
5,chrome_sandbox_t,None,"{sigkill, signal, getsched, sigstop, execmem, setsched, fork, signull, execstack, setrlimit, getcap, sigchld}"
6,condor_startd_t,None,"{sigkill, signal, getsched, sigstop, dyntransition, execmem, transition, fork, signull, getcap, sigchld}"
7,container_t,None,"{sigkill, signal, getsched, sigstop, setcap, getattr, execmem, setsched, getcap, setpgid, fork, signull, execstack, setrlimit, getpgid, sigchld}"
8,couchdb_t,None,"{sigkill, signal, getsched, sigstop, execmem, setsched, fork, signull, getcap, sigchld}"
9,fsadm_t,None,"{sigkill, getsched, noatsecure, getcap, siginh, rlimitinh, setcap, setkeycreate, setsockcreate, setpgid, signull, execstack, getpgid, sigchld, signal, getsession, fork, share, sigstop, dyntransition, getattr, setsched, transition}"


## SELinux

The ability to load policy, toggle enforcement, and change booleans is obviously important for system protection.

In [49]:
p.domains_with(target_name="", tclass=["security"], perms=["load_policy", "setbool", "setenforce"], expand_attrs=True)

,type,conditional,perms
0,init_t,None,"{compute_create, compute_av, compute_user, load_policy, check_context}"
1,abrt_handle_event_t,secure_mode_policyload,{setenforce}
2,anaconda_t,secure_mode_policyload,{setenforce}
3,authconfig_t,secure_mode_policyload,{setenforce}
4,bacula_unconfined_script_t,secure_mode_policyload,{setenforce}
5,boinc_project_t,secure_mode_policyload,{setenforce}
6,bootloader_t,secure_mode_policyload,{setenforce}
7,certmonger_unconfined_t,secure_mode_policyload,{setenforce}
8,cinder_api_t,secure_mode_policyload,{setenforce}
9,cinder_backup_t,secure_mode_policyload,{setenforce}
